In [1]:
import pandas as pd
import psycopg2
from psycopg2 import sql

# AWS RDS Connection

In [2]:
PGEND_POINT = '' # End_point
PGDATABASE_NAME = 'youtube_test_db' # Database Name example: youtube_test_db
PGUSER_NAME = 'postgres' # UserName
PGPASSWORD = 'asd123qwe' # Password

In [3]:
def connect():
    
    # Set up a connection to the postgres server.
    conn_string = "host="+ PGEND_POINT +" port="+ "5432" +" dbname="+ PGDATABASE_NAME +" user=" + PGUSER_NAME \
                  +" password="+ PGPASSWORD
    
    conn = psycopg2.connect(conn_string)
    print("Connected!")

    # Create a cursor object
    cursor = conn.cursor()
    
    return conn, cursor

In [4]:
conn, cursor = connect()

OperationalError: could not translate host name "port=5432" to address: Unknown host


 ## SQL Queries
 

In [ ]:
# Creating simple table
query_food = sql.SQL("""CREATE TABLE Menu (
  food_id INT NOT NULL,
  food_name VARCHAR(30) NOT NULL,
  PRIMARY KEY (food_id),
  UNIQUE (food_name)
);""")



In [ ]:
cur = conn.cursor()
cur.execute(query_food)
cur.close() # close communication with the PostgreSQL database server
conn.commit() # commit the changes

In [ ]:
# insert data into table
query_food_insert = sql.SQL("""
INSERT INTO Menu 
    (food_id, food_name) VALUES (1,'Tacos'), (2,'Tomato Soup'),(3,'Grilled Cheese');
""")

In [ ]:
cur = conn.cursor()
cur.execute(query_food_insert)
cur.close() 
conn.commit()

In [ ]:
# Query the table

query_menu = sql.SQL("""
SELECT * FROM menu;
""")

In [ ]:
cur = conn.cursor()
cur.execute(query_menu)


In [ ]:
print(cur.fetchone())

In [ ]:
cur = conn.cursor()
cur.execute(query_menu)
print(cur.fetchall())

In [ ]:
cur = conn.cursor()
cur.execute(query_menu)
rows_to_fetch = 2
print(cur.fetchmany(rows_to_fetch))

In [ ]:
cur.close() 
conn.commit()

### DataFrame

In [ ]:
cur = conn.cursor()
cur.execute(query_menu)

In [ ]:
df_menu = pd.DataFrame(cur.fetchall(), columns=['food_id', 'food_name'])

In [ ]:
df_menu

In [ ]:
cur.close() 
conn.commit()

#### Insert dataframe into table

In [ ]:
data = {'Name': ['Tom', 'Jerry', 'harry'],
        'Age': [50, 25, 10]}

In [ ]:
df_people = pd.DataFrame(data)

In [ ]:
df_people

In [ ]:
query_people = sql.SQL("""CREATE TABLE people (
  Name VARCHAR(30) NOT NULL,
  Age INT NOT NULL,
  PRIMARY KEY (Name),
  UNIQUE (Name)
);""")

In [ ]:
cur = conn.cursor()
cur.execute(query_people)
cur.close()
conn.commit()

In [ ]:
# single insert
def single_inserts(conn, df, table):
    for i in df.index:
        cur = conn.cursor()
        cols  = ','.join(list(df.columns))
        vals  = [df.at[i,col] for col in list(df.columns)]
        query = "INSERT INTO %s(%s) VALUES('%s',%s)" % (table, cols, vals[0], vals[1])
        cur.execute(query)
        cur.close()
        conn.commit()
    print("single insert completed")


In [ ]:
single_inserts(conn,df_people,'people')

In [ ]:
query_people_table = sql.SQL("""
SELECT * FROM people;
""")

In [ ]:
cur = conn.cursor()
cur.execute(query_people_table)
print(cur.fetchall())
cur.close()
conn.commit()

In [ ]:
# batch insert

In [ ]:
apple_df = pd.read_csv('AAPL.csv') # import dataset

In [ ]:
apple_df.shape

In [ ]:
apple_df.sample(5)

In [ ]:
query_apple = sql.SQL("""CREATE TABLE apple_table (
  Date DATE NOT NULL,
  Open FLOAT,
  High FLOAT,
  Low FLOAT,
  Close FLOAT,
  PRIMARY KEY (Date),
  UNIQUE (Date)
);""")

In [ ]:
cur = conn.cursor()
cur.execute(query_apple)
cur.close()
conn.commit()

In [ ]:
def update_many(conn, datafrm, table):
    
    # Creating a list of tupples from the dataframe values
    tpls = [tuple(x) for x in datafrm.to_numpy()]
    
    # dataframe columns with Comma-separated
    cols = ','.join(list(datafrm.columns))
    
    # SQL query to execute
    sql = "INSERT INTO %s(%s) VALUES(%%s,%%s,%%s,%%s,%%s)" % (table, cols)
    cursor = conn.cursor()
    cursor.executemany(sql, tpls)
    print("Data upload complete successfully...")
    cursor.close()
    conn.commit()

In [ ]:
update_many(conn,apple_df,'apple_table')

In [ ]:
query_apple = sql.SQL("""
SELECT * FROM apple_table;
""")

In [ ]:
cur = conn.cursor()
cur.execute(query_apple)
print(cur.fetchall())
cur.close()
conn.commit()

In [ ]:
cur = conn.cursor()
cur.execute(query_apple)
df_new_apple = pd.DataFrame(cur.fetchall(), columns=['Date', 'Open','High','Low','Close'])

In [ ]:
df_new_apple.head()

In [ ]:
df_new_apple.shape

In [ ]:
apple_df.shape